Serial Schedule Generation Scheme (SGS)

The solver iteratively schedules jobs that are ready (all predecessors completed). For each ready job, it determines the earliest feasible start time by considering:

* Precedence Constraints: Must start after all predecessors finish.
* Resource Constraints: Must start when sufficient renewable and non-renewable resources are available for its entire duration.
The algorithm maintains resource usage over time and incrementally finds the first available time slot that satisfies all constraints.

Heuristics Employed:

* Job Prioritization: When multiple jobs are ready, jobs are prioritized by their duration (Shortest Processing Time first) as a tie-breaker.
* Earliest Feasible Start Time: The solver searches for the absolute earliest time a job can begin by incrementally checking time steps from its earliest possible start time until resource availability is confirmed.
* Resource Consumption: Non-renewable resources are consumed upfront, while renewable resources are consumed for the job's duration.

In [1]:
import json
import time
from collections import defaultdict

def solve_rcpsp(data):
    start_time_perf = time.perf_counter()
    
    jobs_raw = data.get("jobs", [])
    resources_raw = data.get("resources", [])
    
    # 1. Resource Setup with Type Normalization
    renewable_caps = {}
    non_renewable_stocks = {}
    
    for r in resources_raw:
        r_id = str(r["id"]) # Normalize ID to string
        if "capacity" in r:
            renewable_caps[r_id] = r["capacity"]
        if "initial_stock" in r:
            non_renewable_stocks[r_id] = r["initial_stock"]

    # 2. Job Setup
    jobs = {}
    predecessors = defaultdict(list)
    successors = defaultdict(list)
    in_degree = {}

    for j in jobs_raw:
        j_id = j["id"]
        # Normalize resource keys to strings to match resource definitions
        req_ren = {str(k): v for k, v in j.get("resources_required", {}).items()}
        req_non = {str(k): v for k, v in j.get("resource_consumption", {}).items()}
        
        jobs[j_id] = {
            "duration": j["duration"],
            "req_ren": req_ren,
            "req_non": req_non
        }
        succs = j.get("precedences", {}).get("time_successors", [])
        successors[j_id] = succs
        for s in succs:
            predecessors[s].append(j_id)

    all_job_ids = list(jobs.keys())
    for j_id in all_job_ids:
        in_degree[j_id] = len(predecessors[j_id])

    # 3. Scheduling Loop
    scheduled = {}
    finished_time = {}
    ren_usage = defaultdict(lambda: defaultdict(int))
    eligible = [j_id for j_id in all_job_ids if in_degree[j_id] == 0]
    
    while len(scheduled) < len(all_job_ids):
        found_job = False
        # Sort by duration (Shortest Processing Time) as a tie-breaker
        eligible.sort(key=lambda x: (jobs[x]["duration"], x))
        
        for i, j_id in enumerate(eligible):
            job_data = jobs[j_id]
            
            # --- Robust Non-Renewable Check ---
            can_afford_non_ren = True
            temp_requirements = {} # Store parsed amounts to avoid re-calculating
            
            for r_id, req in job_data["req_non"].items():
                amt = req if isinstance(req, (int, float)) else list(req.values())[0]
                temp_requirements[r_id] = amt
                
                # Check if resource exists; if not, it can't be afforded unless req is 0
                if r_id not in non_renewable_stocks:
                    if amt > 0:
                        can_afford_non_ren = False
                        break
                elif non_renewable_stocks[r_id] < amt:
                    can_afford_non_ren = False
                    break
            
            if not can_afford_non_ren:
                continue # Skip this job for now

            # --- Timing and Renewable Check ---
            earliest_start = max([finished_time[p] for p in predecessors[j_id]] + [0])
            current_t = earliest_start
            
            while True:
                fit = True
                for t in range(current_t, current_t + job_data["duration"]):
                    for r_id, req in job_data["req_ren"].items():
                        amt = req if isinstance(req, (int, float)) else list(req.values())[0]
                        if ren_usage[t][r_id] + amt > renewable_caps.get(r_id, 0):
                            fit = False; break
                    if not fit: break
                
                if fit:
                    # SUCCESS: Commit Schedule
                    scheduled[j_id] = current_t
                    finished_time[j_id] = current_t + job_data["duration"]
                    
                    # Deduct Non-Renewables
                    for r_id, amt in temp_requirements.items():
                        non_renewable_stocks[r_id] -= amt
                    
                    # Update Renewable Profile
                    for t in range(current_t, current_t + job_data["duration"]):
                        for r_id, req in job_data["req_ren"].items():
                            amt = req if isinstance(req, (int, float)) else list(req.values())[0]
                            ren_usage[t][r_id] += amt
                    
                    eligible.pop(i)
                    for succ in successors[j_id]:
                        in_degree[succ] -= 1
                        if in_degree[succ] == 0:
                            eligible.append(succ)
                    found_job = True
                    break
                else:
                    current_t += 1
            
            if found_job: break
        
        if not found_job: # Safety break for infeasible problems
            break

    return {
        "schedule": scheduled,
        "metrics": {
            "makespan": max(finished_time.values()) if finished_time else 0,
            "time": (time.perf_counter() - start_time_perf)*1000
        }
    }

In [3]:
with open("C:/Users/Admin/Desktop/nonrenewable.json", 'r') as f:
    example_input = json.load(f)  

    
result = solve_rcpsp(example_input)
print(json.dumps(result, indent=2))

{
  "schedule": {
    "0": 0,
    "1": 0,
    "13": 0,
    "25": 0,
    "37": 0,
    "49": 0,
    "15": 0,
    "38": 0,
    "51": 0,
    "6": 0,
    "14": 0,
    "16": 1,
    "29": 0,
    "41": 0,
    "3": 0,
    "46": 0,
    "2": 0,
    "9": 4,
    "5": 4,
    "10": 4,
    "28": 0,
    "35": 0,
    "40": 0,
    "50": 0,
    "52": 4,
    "56": 9,
    "57": 4,
    "4": 2,
    "7": 9,
    "11": 9,
    "17": 1,
    "23": 8,
    "54": 0,
    "55": 7,
    "8": 9,
    "12": 17,
    "26": 0,
    "34": 8,
    "31": 8,
    "27": 8,
    "33": 15,
    "30": 14,
    "42": 0,
    "43": 8,
    "19": 8,
    "20": 17,
    "32": 15,
    "36": 24,
    "47": 1,
    "53": 10,
    "58": 19,
    "59": 11,
    "60": 22,
    "18": 22,
    "21": 32,
    "22": 42,
    "24": 45,
    "39": 2,
    "44": 16,
    "45": 12,
    "48": 19,
    "61": 45,
    "62": 45,
    "74": 45,
    "77": 45,
    "78": 45,
    "75": 45,
    "63": 45,
    "76": 45,
    "79": 50,
    "64": 45,
    "65": 53,
    "66": 53,
    "80": 50,
